
***Trafikverkets API nyckel*** ---
Namn: `labb2nyckel1`
Nyckel: `--`

In [0]:
pip install azure-cosmos pandas requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/80.8 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 71.7/80.8 kB 2.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/303.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 297.0/303.2 kB 9.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.2/303.2 kB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import requests
import pandas as pd
from datetime import datetime
from azure.cosmos import CosmosClient, PartitionKey

In [0]:
# -----------------------------
# 1. Hämta data från Trafikverket (samma som din kod)
# -----------------------------
url = "https://api.trafikinfo.trafikverket.se/v2/data.json"

# XML Declaration ("this document uses UTF-8 encoding")
body = """<?xml version="1.0" encoding="UTF-8"?>
<REQUEST>
  <LOGIN authenticationkey="---" />
  <QUERY objecttype="TrafficFlow" namespace="road.trafficinfo" schemaversion="1.5" limit="10">
    <FILTER>
      <GT name="MeasurementTime" value="$dateadd(-0.00:15:00)" />
      <EQ name="RegionId" value="4" />
    </FILTER>
  </QUERY>
</REQUEST>"""

# HTTP Header ("the HTTP body bytes are encoded as UTF-8")
headers = {"Content-Type": "application/xml; charset=utf-8"}

# Python Encoding ("Converts Python string → UTF-8 bytes")
encoded_body = body.encode("utf-8")
response = requests.post(url, data=encoded_body, headers=headers)
response.raise_for_status()
 
data = response.json()
flows = data["RESPONSE"]["RESULT"][0]["TrafficFlow"]
 
df = pd.DataFrame(flows)

In [0]:
# Check if successful
print(f"Status: {response.status_code}")

# Get response content
xml_response = response.text  # as string
xml_bytes = response.content # as bytes

# print(f"Status: {xml_response}")
# print(f"Status: {xml_bytes}")

Status: 200


In [0]:
df

,SiteId,MeasurementTime,MeasurementOrCalculationPeriod,VehicleType,VehicleFlowRate,AverageVehicleSpeed,CountyNo,Deleted,Geometry,RegionId,DataQuality,SpecificLane,MeasurementSide,ModifiedTime
0,40,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1200,68.55,1,False,"{'SWEREF99TM': 'POINT (677754.96 6578623.19)',...",4,good,lane1,unknown,2025-09-30T16:25:07.513Z
1,4306,2025-09-30T18:25:00.000+02:00,60,anyVehicle,660,73.31,1,False,"{'SWEREF99TM': 'POINT (674788.02 6583302.84)',...",4,good,lane2,unknown,2025-09-30T16:25:07.513Z
2,861,2025-09-30T18:25:00.000+02:00,60,anyVehicle,600,96.70,1,False,"{'SWEREF99TM': 'POINT (651018.96 6563771.14)',...",4,good,lane2,unknown,2025-09-30T16:25:07.835Z
3,1471,2025-09-30T18:25:00.000+02:00,60,anyVehicle,780,81.66,1,False,"{'SWEREF99TM': 'POINT (652805.94 6565755.17)',...",4,good,lane2,unknown,2025-09-30T16:25:07.781Z
4,174,2025-09-30T18:25:00.000+02:00,60,anyVehicle,720,85.53,1,False,"{'SWEREF99TM': 'POINT (656154.96 6566753.11)',...",4,good,lane1,unknown,2025-09-30T16:25:04.873Z
5,1183,2025-09-30T18:25:00.000+02:00,60,anyVehicle,840,59.21,1,False,"{'SWEREF99TM': 'POINT (673561 6576993.92)', 'W...",4,good,lane1,unknown,2025-09-30T16:25:07.514Z
6,661,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1320,35.14,1,False,"{'SWEREF99TM': 'POINT (673560.02 6577001.1)', ...",4,good,lane3,unknown,2025-09-30T16:25:07.676Z
7,1155,2025-09-30T18:25:00.000+02:00,60,anyVehicle,300,68.76,1,False,"{'SWEREF99TM': 'POINT (674026.99 6576474.18)',...",4,good,lane1,unknown,2025-09-30T16:25:07.728Z
8,797,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1440,58.54,1,False,"{'SWEREF99TM': 'POINT (673338.96 6577008.01)',...",4,good,lane2,unknown,2025-09-30T16:25:04.978Z
9,614,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1140,71.26,1,False,"{'SWEREF99TM': 'POINT (672453 6577193.9)', 'WG...",4,good,lane2,unknown,2025-09-30T16:25:05.087Z


In [0]:
type(df)

pandas.core.frame.DataFrame

In [0]:
# -----------------------------
# 1.1. Lagra hämtad data i rådata
# -----------------------------
account   = "labb1bi24aroos"                         
container = "imports"                          
sas       = "sp=rcwdl&st=2025-09-30T15:12:59Z&se=2025-10-13T23:27:59Z&spr=https&sv=2024-11-04&sr=c&sig=YNuY7fqJ26sj7PZ9ZK8WYVSHC9Un7h%2B2AGnyOpHNo6o%3D" # SAS-token 
#input_rel = "Kontaktuppgifter_kommuner_20250417_unicode.csv"   # filnamnet i containern

# Sätt konfiguration för att använda SAS-token
spark.conf.set(f"fs.azure.sas.{container}.{account}.blob.core.windows.net", sas)

# Bas-URL
base_path = f"wasbs://{container}@{account}.blob.core.windows.net/api_data/"

# "overwrite" eller "append" om du vill lägga till ny data
spark_df = spark.createDataFrame(df)
spark_df.write.format("parquet") \
    .mode("overwrite") \
    .save(base_path)

display(spark_df.limit(10))

SiteId,MeasurementTime,MeasurementOrCalculationPeriod,VehicleType,VehicleFlowRate,AverageVehicleSpeed,CountyNo,Deleted,Geometry,RegionId,DataQuality,SpecificLane,MeasurementSide,ModifiedTime
40,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1200,68.55,1,false,"List(POINT (677754.96 6578623.19), POINT (18.122711 59.308964))",4,good,lane1,unknown,2025-09-30T16:25:07.513Z
4306,2025-09-30T18:25:00.000+02:00,60,anyVehicle,660,73.31,1,false,"List(POINT (674788.02 6583302.84), POINT (18.074469 59.352165))",4,good,lane2,unknown,2025-09-30T16:25:07.513Z
861,2025-09-30T18:25:00.000+02:00,60,anyVehicle,600,96.7,1,false,"List(POINT (651018.96 6563771.14), POINT (17.64332 59.186134))",4,good,lane2,unknown,2025-09-30T16:25:07.835Z
1471,2025-09-30T18:25:00.000+02:00,60,anyVehicle,780,81.66,1,false,"List(POINT (652805.94 6565755.17), POINT (17.675951 59.203293))",4,good,lane2,unknown,2025-09-30T16:25:07.781Z
174,2025-09-30T18:25:00.000+02:00,60,anyVehicle,720,85.53,1,false,"List(POINT (656154.96 6566753.11), POINT (17.735239 59.211025))",4,good,lane1,unknown,2025-09-30T16:25:04.873Z
1183,2025-09-30T18:25:00.000+02:00,60,anyVehicle,840,59.21,1,false,"List(POINT (673561 6576993.92), POINT (18.047848 59.296097))",4,good,lane1,unknown,2025-09-30T16:25:07.514Z
661,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1320,35.14,1,false,"List(POINT (673560.02 6577001.1), POINT (18.047836 59.29616))",4,good,lane3,unknown,2025-09-30T16:25:07.676Z
1155,2025-09-30T18:25:00.000+02:00,60,anyVehicle,300,68.76,1,false,"List(POINT (674026.99 6576474.18), POINT (18.0556 59.291245))",4,good,lane1,unknown,2025-09-30T16:25:07.728Z
797,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1440,58.54,1,false,"List(POINT (673338.96 6577008.01), POINT (18.043966 59.296314))",4,good,lane2,unknown,2025-09-30T16:25:04.978Z
614,2025-09-30T18:25:00.000+02:00,60,anyVehicle,1140,71.26,1,false,"List(POINT (672453 6577193.9), POINT (18.028582 59.298344))",4,good,lane2,unknown,2025-09-30T16:25:05.087Z


In [0]:
# -----------------------------
# 2. Konfigurera anslutning till Cosmos DB
# -----------------------------
endpoint = "https://labb2-cosmos-db.documents.azure.com:443/"
key = "---"
 
# Skapa Cosmos DB-klient
client = CosmosClient(endpoint, key)
 
# Välj eller skapa databas
database_name = "traffic"
database = client.create_database_if_not_exists(id=database_name)
 
# Välj eller skapa container 
container_name = "traffic"
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path="/partitionKey")
)

In [0]:
# -----------------------------
# 3. Konvertera DataFrame till JSON och lägg till id och tidsstämpel
# -----------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
 
# Konvertera DataFrame till lista av JSON-dokument
records = df.to_dict(orient="records")
 
# Lägg till unik id och tidsstämpel för varje dokument
for record in records:
    record["id"] = f"{record.get('MeasurementTime', 'unknown')}_{timestamp}_{hash(str(record))}"  # Unik id
    record["timestamp"] = timestamp  # Lägg till tidsstämpel

In [0]:
# -----------------------------
# 4. Spara data till Cosmos DB
# -----------------------------
container_name = "traffic"

for record in records:
    container.upsert_item(record)
 
print(f"Data har laddats upp till Cosmos DB i container: {container_name}")

Data har laddats upp till Cosmos DB i container: traffic


In [0]:
## Test lagring

account   = "labb1bi24aroos"                         
container = "imports"                          
sas       = "sp=rcwdl&st=2025-09-25T20:39:53Z&se=2025-10-17T04:54:53Z&spr=https&sv=2024-11-04&sr=c&sig=CVgPIflrA55n9mSOmj16eh5sqVdLgbTgPT8oysFggUE%3D"    # SAS-token du genererade
input_rel = "Kontaktuppgifter_kommuner_20250417_unicode.csv"   # filnamnet i containern
# Sätt konfiguration för att använda SAS-token
spark.conf.set(f"fs.azure.sas.{container}.{account}.blob.core.windows.net", sas)
# Bas-URL
base = f"wasbs://{container}@{account}.blob.core.windows.net/"
# Läs in filen
df = (spark.read
     .option("header", True)        # sätt till False om filen inte har rubrikrad
     .option("inferSchema", True)
     .option("delimiter", ";")     # byt till "," om det är CSV, "\t" för tab-separerad
     .csv(base + input_rel))
display(df.limit(10))

Läns-/kommunkod,Länstillhörighet,Organisationsnummer,Namn,E-post,Telefon,Webbadress,Postadress 1,Postnr,Ort
1440,14,212000-1439,Ale kommun,kommun@ale.se,0303-70 30 00,http://www.ale.se,null,44980,ALAFORS
1489,14,212000-1553,Alingsås kommun,kommunstyrelsen@alingsas.se,0322-616000,http://www.alingsas.se,null,44181,ALINGSÅS
764,7,212000-0639,Alvesta kommun,kommunen@alvesta.se,0472-15000,http://www.alvesta.se,null,34280,ALVESTA
604,6,212000-0498,Aneby kommun,info@aneby.se,0380-461 00,http://www.aneby.se,Box 53,57822,ANEBY
1984,19,212000-2122,Arboga kommun,arboga.kommun@arboga.se,0589-87000,http://www.arboga.se,Box 45,73221,ARBOGA
2506,25,212000-2668,Arjeplogs kommun,kommun@arjeplog.se,0961-140 00,http://www.arjeplog.se,null,93881,ARJEPLOG
2505,25,212000-2650,Arvidsjaurs kommun,kommun@arvidsjaur.se,0960-15500,http://www.arvidsjaur.se,null,93381,ARVIDSJAUR
1784,17,212000-1892,Arvika kommun,arvika.kommun@arvika.se,0570-816 00,http://www.arvika.se,null,67181,ARVIKA
1882,18,212000-1983,Askersunds kommun,kommun@askersund.se,0583-81000,http://www.askersund.se,null,69682,ASKERSUND
2084,20,212000-2262,Avesta kommun,kommun@avesta.se,0226-645000,http://www.avesta.se,null,77481,AVESTA
